In [50]:
import cv2
import nibabel as nib
import numpy as np
from pycimg import CImg

In [51]:
IMG_NUM = 10

fname = './Images/IMG_%04d.nii.gz' % IMG_NUM
lungs = nib.load(fname).get_fdata()
fname = './BodyMasks/BODYMASK_IMG_%04d.nii.gz' % IMG_NUM
bodymask_gt = nib.load(fname).get_fdata()
max_val = np.max(lungs)

lungs_bin_inv = np.zeros(lungs.T.shape)
for i, dim in enumerate(lungs.T):
    _, lungs_bin_inv[i] = cv2.threshold(dim, -320, max_val, cv2.THRESH_BINARY_INV)

lungs_bin_inv = lungs_bin_inv.T
# CImg(lungs_bin_inv).display();


In [52]:
def reconstruct(marker: np.ndarray, mask: np.ndarray, radius: int = 1):
    kernel = np.ones(shape=(radius * 2 + 1,) * 2, dtype=np.uint8)
    while True:
        expanded = cv2.dilate(src=marker, kernel=kernel)
        cv2.bitwise_and(src1=expanded, src2=mask, dst=expanded)
        if (marker == expanded).all():
            return expanded
        
        marker = expanded

In [53]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
bodymask_pred = np.zeros(lungs.T.shape)
for i, dim in enumerate(lungs.T):
    _, bin = cv2.threshold(dim, -200, max_val, cv2.THRESH_BINARY)
    img = cv2.erode(bin, kernel, iterations=1)
    img = cv2.dilate(img, kernel, iterations=1)
    img_neg = np.logical_not(img).astype(np.uint8)
    border_marker = np.zeros(img.shape)
    border_marker[0] = 1
    border_marker[-1] = 1
    border_marker[:, 0] = 1
    border_marker[:, -1] = 1
    border_marker = np.logical_and(img_neg, border_marker).astype(np.uint8)
    reconstructed = reconstruct(border_marker, img_neg)
    cleared_border = img_neg - reconstructed
    filled_img = np.logical_or(cleared_border, img).astype(np.uint8)
    bodymask_pred[i] = filled_img

bodymask_pred = bodymask_pred.T
CImg(bodymask_pred).display();

In [54]:
lungs_air_within_body = np.logical_and(lungs_bin_inv, bodymask_pred).astype(np.uint8)
CImg(lungs_air_within_body).display();

In [55]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
lungs_morphed = np.zeros(lungs_air_within_body.T.shape)
for i, dim in enumerate(lungs_air_within_body.T):
    lungs_morphed[i] = cv2.medianBlur(dim, 7)

lungs_morphed = lungs_morphed.T

lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.erode(lungs_morphed, kernel, None, iterations=2)
lungs_morphed = cv2.dilate(lungs_morphed, kernel, None, iterations=2)
CImg(lungs_morphed).display()


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [56]:
lungs_processed = np.zeros(lungs_morphed.T.shape)
for i, dim in enumerate(lungs_morphed.T):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(dim.astype(np.uint8))
    for j in range(num_labels):
        if stats[j][cv2.CC_STAT_AREA] < 500:
            labels[labels == j] = 0

    lungs_processed[i] = labels

print(num_labels)
for i, dim in enumerate(lungs_processed):
    _, lungs_processed[i] = cv2.threshold(dim, 0, 255, cv2.THRESH_BINARY)

lungs_processed = lungs_processed.T.astype(np.uint8)
CImg(lungs_processed).display()

2


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [57]:
from skimage.segmentation import watershed
from scipy import ndimage as ndi
from skimage import filters

boundaries = np.zeros(lungs_processed.T.shape)
gradients = np.zeros(lungs_processed.T.shape)
sure_fg_3D = np.zeros(lungs_processed.T.shape)
for i, processed_img in enumerate(lungs_processed.T):
    gradients[i] = filters.sobel(processed_img)
    sure_fg = cv2.erode(processed_img, np.ones((15, 15)), iterations=3).astype(np.uint8) # tu jest problem. musi być aż 15, bo w niektórych przykładach te płuca są bardzo połączone (np. 7). Ale to trochę psuje wyniki, można to jakoś poprawić...
    sure_bg = cv2.dilate(processed_img, np.ones((7, 7)), iterations=3).astype(np.uint8)
    boundaries[i] = cv2.subtract(sure_bg, sure_fg)
    sure_fg_3D[i] = sure_fg

boundaries = boundaries.T
sure_fg_3D = sure_fg_3D.T
markers, num_labels = ndi.label(sure_fg_3D)
markers += 1
markers[boundaries == 255] = 0
labels_sums = np.zeros(num_labels)
for i in range(num_labels):
    labels_sums[i] = np.sum((markers==i)*1)

top_arg =  np.argmax(labels_sums)
print(top_arg)
trimmed_labels = np.zeros(markers.shape)
trimmed_labels += (markers == top_arg) * 1 # background
for i in range(1, 4):
    labels_sums[top_arg] = 0
    top_arg = np.argmax(labels_sums)
    print(top_arg)
    if top_arg != 0: # not border marker
        trimmed_labels += (markers == top_arg) * i

markers = trimmed_labels.astype(np.int32)
CImg(markers).display()

gradients /= np.max(gradients)
gradients *= 255
gradients = gradients.T.astype(np.int32)
CImg(gradients).display()


1
0
2
4


CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [63]:
lungs_done = watershed(gradients, markers)
CImg(lungs_done).display()

CImg(array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],

In [62]:
fname = './ReferenceSegmentations/LUNGS_IMG_%04d.nii.gz' % IMG_NUM
lungs_test = nib.load(fname).get_fdata()
CImg(lungs_test).display()

CImg(array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],

In [60]:
import surface_distance

left_lung_pred = lungs_done == 2
right_lung_pred = lungs_done == 3

left_lung_gt = lungs_test == 2
right_lung_gt = lungs_test == 3

dists_ideal = surface_distance.compute_surface_distances(left_lung_gt, left_lung_gt, (1, 1, 1))
hausdorff_ideal = surface_distance.compute_robust_hausdorff(dists_ideal, 100)
dice_vol_coef_ideal = surface_distance.compute_dice_coefficient(left_lung_gt, left_lung_gt)

dists_left = surface_distance.compute_surface_distances(left_lung_gt, left_lung_pred, (1, 1, 1))
hausdorff_left = surface_distance.compute_robust_hausdorff(dists_left, 100)
dice_vol_coef_left = surface_distance.compute_dice_coefficient(left_lung_gt, left_lung_pred)

dists_right = surface_distance.compute_surface_distances(right_lung_gt, right_lung_pred, (1, 1, 1))
hausdorff_right = surface_distance.compute_robust_hausdorff(dists_right, 100)
dice_vol_coef_right = surface_distance.compute_dice_coefficient(right_lung_gt, right_lung_pred)

print(hausdorff_ideal, dice_vol_coef_ideal)
print(hausdorff_left, dice_vol_coef_left)
print(hausdorff_right, dice_vol_coef_right)


0.0 1.0
175.0571335307419 0.0
173.3955016717562 0.0
